In [123]:
import pandas as pd

# Load original datasets
crashes_df = pd.read_excel(r"D:\UWA\Data_Warehousing\Project 1\bitre_fatal_crashes_dec2024.xlsx", sheet_name="BITRE_Fatal_Crash", header=4)
fatalities_df = pd.read_excel(r"D:\UWA\Data_Warehousing\Project 1\bitre_fatalities_dec2024.xlsx", sheet_name="BITRE_Fatality", header=4)

# Show first 5 rows
print(crashes_df.head(5))
print(fatalities_df.head(5))



   Crash ID State  Month  Year Dayweek      Time Crash Type  \
0  20241115   NSW     12  2024  Friday  04:00:00     Single   
1  20241125   NSW     12  2024  Friday  06:15:00     Single   
2  20246013   Tas     12  2024  Friday  09:43:00   Multiple   
3  20241002   NSW     12  2024  Friday  10:35:00   Multiple   
4  20242261   Vic     12  2024  Friday  11:30:00   Multiple   

   Number Fatalities Bus \nInvolvement Heavy Rigid Truck Involvement  \
0                  1                No                            No   
1                  1                No                            No   
2                  1                No                            No   
3                  1                No                            No   
4                  1                -9                            -9   

  Articulated Truck Involvement Speed Limit National Remoteness Areas  \
0                            No         100  Inner Regional Australia   
1                            No          8

In [124]:
# Rename "Bus \nInvolvement" column to "Bus Involvement"
crashes_df.rename(columns={"Bus \nInvolvement": "Bus Involvement"}, inplace=True)

In [125]:
print(crashes_df.shape)
print(fatalities_df.shape)

(51284, 20)
(56874, 23)


In [126]:
# Check data
crashes_df.describe()
fatalities_df.describe()

,Crash ID,Month,Year,Age
count,5.687400e+04,56874.000000,56874.000000,56874.000000
mean,2.007641e+07,6.596142,2004.158086,40.036414
std,2.410814e+06,3.465712,10.417546,21.891551
min,1.989100e+07,1.000000,1989.000000,-9.000000
25%,1.995308e+07,4.000000,1995.000000,22.000000
50%,2.003223e+07,7.000000,2003.000000,35.000000
75%,2.013112e+07,10.000000,2013.000000,56.000000
max,2.018501e+08,12.000000,2024.000000,101.000000


In [127]:
# absolute NaN counts as well
print("\n🔢 Total NaN in Crashes:")
print(crashes_df.isna().sum()[crashes_df.isna().sum() > 0].sort_values(ascending=False))

print("\n🔢 Total NaN in Fatalities:")
print(fatalities_df.isna().sum()[fatalities_df.isna().sum() > 0].sort_values(ascending=False))

# Since these columns are not useful, we can ignore 


🔢 Total NaN in Crashes:
SA4 Name 2021             39579
National LGA Name 2021    39578
Time                         39
dtype: int64

🔢 Total NaN in Fatalities:
SA4 Name 2021             44175
National LGA Name 2021    44173
Time                         43
dtype: int64


In [128]:
# Replace common invalids with NaN - these invalid values has been noticed in data
invalids = ['-9', -9, 'Unknown', '', ' ']
crashes_df.replace(invalids, pd.NA, inplace=True)
fatalities_df.replace(invalids, pd.NA, inplace=True)

In [129]:
# Show % of missing data (only > 0%)
print("\n Missing % in Crashes:")
missing_crash = crashes_df.isna().mean().round(4) * 100
print(missing_crash[missing_crash > 0].sort_values(ascending=False))

print("\n Missing % in Fatalities:")
missing_fatal = fatalities_df.isna().mean().round(4) * 100
print(missing_fatal[missing_fatal > 0].sort_values(ascending=False))


 Missing % in Crashes:
SA4 Name 2021                    80.18
National LGA Name 2021           80.17
National Remoteness Areas        79.58
Heavy Rigid Truck Involvement    35.51
Speed Limit                       2.60
Bus Involvement                   0.12
Articulated Truck Involvement     0.11
Time                              0.08
Time of Day                       0.08
Day of week                       0.02
dtype: float64

 Missing % in Fatalities:
SA4 Name 2021                    80.62
National LGA Name 2021           80.62
National Remoteness Areas        80.04
Heavy Rigid Truck Involvement    36.14
Speed Limit                       2.61
Age Group                         0.21
Age                               0.20
Bus Involvement                   0.12
Articulated Truck Involvement     0.11
Time                              0.08
Time of day                       0.08
Gender                            0.06
Road User                         0.02
Day of week                       0.0

In [130]:
# Drop rows missing critical fields
crashes_df_clean = crashes_df.dropna(subset=['Time', 'Time of Day', 'Day of week', 'Speed Limit', 'Bus Involvement', 'Articulated Truck Involvement'], inplace=False)
fatalities_df_clean = fatalities_df.dropna(subset=[ 'Road User', 'Time', 'Time of day', 'Gender', 'Day of week', 'Speed Limit', 'Age', 'Age Group'], inplace=False)

In [131]:
# Show % of missing data (only > 0%) after dropping invalid rows
print("\n Missing % in Crashes:")
missing_crash = crashes_df_clean.isna().mean().round(4) * 100
print(missing_crash[missing_crash > 0].sort_values(ascending=False))

print("\n Missing % in Fatalities:")
missing_fatal = fatalities_df_clean.isna().mean().round(4) * 100
print(missing_fatal[missing_fatal > 0].sort_values(ascending=False))

# Now we can see that critical columns don't have invalid records


 Missing % in Crashes:
SA4 Name 2021                    79.83
National LGA Name 2021           79.83
National Remoteness Areas        79.24
Heavy Rigid Truck Involvement    34.91
dtype: float64

 Missing % in Fatalities:
SA4 Name 2021                    80.24
National LGA Name 2021           80.23
National Remoteness Areas        79.66
Heavy Rigid Truck Involvement    35.57
Bus Involvement                   0.09
Articulated Truck Involvement     0.08
dtype: float64


In [132]:
# check % of data loss
crashes_data_loss_per = ((crashes_df.shape[0]-crashes_df_clean.shape[0])*100)/crashes_df.shape[0]
fatalities_data_loss_per = ((fatalities_df.shape[0]-fatalities_df_clean.shape[0])*100)/fatalities_df.shape[0]
print(f"\n Crashes data loss % = { crashes_data_loss_per} %")
print(f" Fatalities data loss % = { fatalities_data_loss_per} %")

### Both are less than 5%


 Crashes data loss % = 2.731846189844786 %
 Fatalities data loss % = 2.8853254562717585 %


In [133]:
# lets overwrite our original dataframes
crashes_df = crashes_df_clean
fatalities_df = fatalities_df_clean

In [134]:
# Step 5: Replace other nulls with 'Unknown'
crashes_df.fillna("Unknown", inplace=True)
fatalities_df.fillna("Unknown", inplace=True)

C:\Users\JHAKU\AppData\Local\Temp\ipykernel_28648\3548154164.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fatalities_df.fillna("Unknown", inplace=True)


In [135]:
# Final shape
print(f"\n Cleaned crashes: {crashes_df.shape[0]} rows and {crashes_df.shape[1]} columns")
print(f" Cleaned fatalities: {fatalities_df.shape[0]} rows and {fatalities_df.shape[1]} columns")


 Cleaned crashes: 49883 rows and 20 columns
 Cleaned fatalities: 55233 rows and 23 columns


In [136]:
# Load data into PostgreSQL using SQLAlchemy
from sqlalchemy import create_engine

DB_USER = 'postgres'
DB_PASS = 'test'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'Project_DW01'

# Create SQLAlchemy engine
engine = create_engine(f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}')
conn=engine.connect()

# Load cleaned data into staging tables
try:
    crashes_df.to_sql('stg_crashes', engine, if_exists='replace', index=False)
    fatalities_df.to_sql('stg_fatalities', engine, if_exists='replace', index=False)
    print("\n✅ Data loaded into PostgreSQL successfully.")
except Exception as e:
    print(f"\n❌ Failed to load into PostgreSQL: {e}")


✅ Data loaded into PostgreSQL successfully.


In [137]:
!pip install psycopg2-binary

In [138]:
# Read dwelling count csv
dwelling_df = pd.read_csv(
    r"D:\UWA\Data_Warehousing\Project 1\LGA (count of dwellings).csv",
    skiprows=11,
    usecols=[0, 1],
    names=['lga_name', 'dwelling_count']
)

dwelling_df.head()

,lga_name,dwelling_count
0,Albury,25430
1,Armidale Regional,12955
2,Ballina,20889
3,Balranald,1091
4,Bathurst Regional,18458


In [139]:
# Check total number of rows and columns
print(dwelling_df.shape)

# Check % of missing data (only > 0%) after dropping invalid rows
print("\n Missing % in Crashes:")
missing_dwelling_count = dwelling_df.isna().mean().round(4) * 100
print(missing_dwelling_count[missing_dwelling_count > 0].sort_values(ascending=False))

(561, 2)

 Missing % in Crashes:
dwelling_count    0.53
dtype: float64


In [140]:
# Since 0.53 is very less, we can simply drop it
dwelling_df.dropna(subset=['lga_name', 'dwelling_count'], inplace=True)

#Check rows number after dropping invalid records
dwelling_df.shape

(558, 2)

In [141]:
# Check for duplicate records
dwelling_df = dwelling_df.dropna().drop_duplicates()
dwelling_df = dwelling_df[dwelling_df['dwelling_count'].apply(lambda x: str(x).isdigit())]
dwelling_df['dwelling_count'] = dwelling_df['dwelling_count'].astype(int)
dwelling_df.shape

(557, 2)

In [142]:
# Load data into staging table
from sqlalchemy import create_engine

DB_USER = 'postgres'
DB_PASS = 'test'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'Project_DW01'

# Create SQLAlchemy engine
engine = create_engine(f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}')
conn=engine.connect()

# Load cleaned data into staging tables
try:
    dwelling_df.to_sql('stg_dwelling_count', engine, if_exists='replace', index=False)
    print("\n Data loaded into PostgreSQL successfully.")
except Exception as e:
    print(f"\n Failed to load into PostgreSQL: {e}")


 Data loaded into PostgreSQL successfully.
